In [1]:
import shared.config as config
import shared.const as const
import pymongo
import numpy as np
import pandas as pd
import json
import re
import ast
import logging

from pymongo import ReplaceOne
import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

In [2]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]

stagingPlaatje = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
stagingDigiFoto = stagingDb[config.COLL_STAGING_DIGIFOTOS]
collection = analyseDb[config.COLL_ANALYSE_CLEAN]

logger = logging.getLogger()


In [3]:
soort = 'Artefact'
soort_lw = soort.lower()
key = 'key_foto3'
refkey = None

In [4]:
#df_soort = pd.DataFrame(list(cleancollection.find({"soort": "Artefact"}, projection={'key_foto1':1, 'ID':1}))) #find(soort_query, projection={key:1, 'ID':1})
#df_soort

soort_query = {'soort': soort, key: {"$exists": True}}
if key == 'key':
    df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))

    key = 'key_' + soort_lw
    df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID', 'key': key})
else:
    df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))
    df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID'})

df_soort

""


In [5]:
df_ref = pd.DataFrame(list(collection.find({key: {"$exists": True}, 'soort': {'$ne': soort}, soort_lw + 'ID': {"$exists": False}}, projection={key:1, 'soort': 1})))
df_ref

""


In [6]:
df_merge = pd.merge(df_ref, df_soort, how='inner', on=key).rename(columns={'ID': (soort_lw if not refkey else refkey) + 'ID'})
df_merge

KeyError: 'key_foto3'

In [ ]:
df2 = df_merge.groupby(['_id']).size().reset_index(name='counts')
df2 = df2[df2.counts == 1]
df2

In [ ]:
df_merge = pd.merge(df2, df_merge, how='left', on='_id')
df_merge

In [ ]:
str = "/input/images/DC005_Anne Gesticht of Anna Convent/L Fotos/H (Objectfoto's)/Hout/DC05_P01_H0001_722_2.JPG"

'Hout/' in str

In [ ]:
df = pd.DataFrame(list(collection.find({'soort': 'Bestand'})))
                                
df[df.rapporttype.notnull()]

In [10]:
def getAnalyseCollection():   
    myclient = pymongo.MongoClient(str(config.MONGO_URI))
    analyseDb = myclient[str(config.DB_ANALYSE)]
    return analyseDb[config.COLL_ANALYSE]

def getAnalyseCleanCollection():   
    myclient = pymongo.MongoClient(str(config.MONGO_URI))
    analyseDb = myclient[str(config.DB_ANALYSE)]
    return analyseDb[config.COLL_ANALYSE_CLEAN]


def setReferences(soort, col='analyseclean', key='key', refkey=None):
    try:
        if (col == 'analyse'):
            collection = getAnalyseCollection()
        elif (col == 'analyseclean'):
            collection = getAnalyseCleanCollection()
        else:
            raise ValueError('Error: Herkent de collectie niet met naam ' + col)
        soort_lw = soort.lower()

        # Find all main entries for type soort
        soort_query = {'soort': soort}
        if soort in const.BESTANDSOORTEN:
            soort_query = {'soort': 'Bestand', 'bestandsoort_XX': soort}
        soort_query[key] = {"$exists": True}

        if key == 'key':
            df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))

            key = 'key_' + soort_lw
            df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID', 'key': key})
        else:
            df_soort = pd.DataFrame(list(collection.find(soort_query, projection={key:1, 'ID':1})))
            df_soort = df_soort.rename(columns={'_id': soort_lw+'UUID', 'index':soort_lw+'ID'})


        if df_soort.size < 1:
            logger.warning("Er zjn geen documents gevonden van het type " +soort)
            return

        if not key in df_soort.columns:
            logger.warning("Kan geen referenties maken voor " +soort + ". Geen Key-veld aanwezig.")
            return

        # Find all references to type soort
        if refkey:
            ref_query = {'key_'+refkey: {"$exists": True}, 'soort': {'$ne': soort}, (soort_lw if not refkey else refkey) + 'ID': {"$exists": False}}
            df_ref = pd.DataFrame(list(collection.find(ref_query, projection={'key_'+refkey:1})))
            df_ref = df_ref.rename(columns={'key_'+refkey: key})
        else:
            ref_query = {key: {"$exists": True}, 'soort': {'$ne': soort}, (soort_lw if not refkey else refkey) + 'ID': {"$exists": False}}
            df_ref = pd.DataFrame(list(collection.find(ref_query, projection={key:1})))
        if df_ref.size < 1:
            logger.warning("Er zjn geen referentie met key_"+soort_lw+" gevonden naar documents van het type " +soort )
            return
            
        # Merge dataframes to connect ID's en UUID's to referencing docs
        df_merge = pd.merge(df_ref, df_soort, how='inner', on=key).rename(columns={'ID': (soort_lw if not refkey else refkey) + 'ID'})
        if key == 'key_foto3':
            df2 = df_merge.groupby(['_id']).size().reset_index(name='counts')
            df2 = df2[df2.counts == 1]
            df_merge = pd.merge(df2, df_merge, how='inner', on='_id')

        if df_merge.size < 1:
            logger.warning("Er zjn geen referentie met "+key+" gevonden naar documents van het type " +soort )
            return

        # Update soort documents 
        updates=[ UpdateOne({'_id':x['_id']}, {'$set':x}) for x in [v.dropna().to_dict() for k,v in df_merge.iterrows()]] # 
        result = collection.bulk_write(updates)

        return result.bulk_api_result
        
    except Exception as err:
        msg = "Onbekende fout bij het aanroepen van een aggregation met melding: " + str(err)
        logger.error(msg)   
        raise Exception(msg) from err
 
    finally:
        collection.database.client.close()

        
        
setReferences('Artefact', key='key_foto3')        

Er zjn geen documents gevonden van het type Artefact


In [15]:
'''
Convenience methodes to retrieve table and project data from bson (non json) fields for pandas. 
mergeFotoInfo uses these values 
'''
regexTable = re.compile(r'\'table\': \'(.*?)\'') # regex to replace Object
regexProject = re.compile(r'\'projectcd\': \'(.*?)\'') # regex to replace Object
def getTable(brondata):  
    try:
        return regexTable.search(str(brondata)).group(1)
    except:
        return None

def getProject(brondata):    
    try:
        return regexProject.search(str(brondata)).group(1)
    except:
        return None


def is_empty(item):
    if not item or item is None or pd.isna(item) or item == "":
        return True
    else: 
        return False

def not_empty(item):
    return not is_empty(item)

'''
Return first non empty value
'''
def firstValue(*kargs):
    for x in kargs:
        if not_empty(x):
            return x
    return None


def mergeFotoinfo():
    logger.info(f"Starting with photo merging")
    try:
        #Aggregate Pipeline
        collection = getAnalyseCollection()
        cleancollection = getAnalyseCleanCollection()

        # get all Photo records
        df_foto = pd.DataFrame(list(collection.find({"soort": "Foto"})))

        # get all Photo descr records
        df_fotobeschr = pd.DataFrame(list(collection.find({"soort": "Fotobeschrijving"})))
        df_fotobeschr["abcd-nr"] = df_fotobeschr.apply(lambda x: x["pad"].split('\\')[-1].lower() if x["pad"] and type(x["pad"]) == str else '', axis=1) 

        df_fotokoppel = pd.DataFrame(list(collection.find({"soort": "Fotokoppel"})))
        df_fotokoppel["abcd-nr"] = df_fotokoppel.apply(lambda x: x["abcd-nr"].lower() if x["abcd-nr"] and type(x["abcd-nr"]) == str else '', axis=1) 

        # Merge dataframes to get a complete Photo-dataframe
        df_merge = df_foto.merge(df_fotokoppel, how="left", right_on="bestandsnaam", left_on="fileName", suffixes=("", "_koppel"))
        df_merge = df_merge.merge(df_fotobeschr, how="left", on=["abcd-nr", "projectcd"], suffixes=("", "_beschr"))
        df_merge['soort'] = 'Bestand'
        df_merge['bestandsoort_XX'] = 'Foto'
        df_merge['brondata'] = df_merge.apply(lambda x: [x['brondata'], x['brondata_beschr']], axis=1)
        df_merge['wasstraat'] = df_merge.apply(lambda x: [{'projectcd': x['projectcd'], 'table': getTable(elem)} for elem in x['brondata'] if getTable(elem)], axis=1)  
        df_merge['materiaal'] = df_merge.apply(lambda x: firstValue(x['materiaal'], x['materiaalgroep']) if 'materiaal' in df_merge.columns else x['materiaalgroep'],axis=1)
        df_merge = df_merge[['_id', 'fileName', 'imageID', 'imageMiddleID', 'imageThumbID',
            'fileType', 'directory', 'mime_type', 'fototype', 'soort', 'projectcd',
            'materiaal', 'putnr', 'vondstnr', 'fotonr', 'vondstkey_met_putnr',
            'key', 'key_project', 'key_project_type', 'key_vondst',
            'key_artefact', 'subnr', 'brondata', 'key_foto1', 'key_foto2', 'key_foto3'
            'pad', 'spoornr', 'profiel', 'subnr', 'datum', 'omschrijving', 'vlaknr', 'richting', 'wasstraat', 'bestandsoort', 'bestandsoort_XX']]


        #updates= [ pymongo.ReplaceOne({"_id": record['_id']}, record, upsert=True) for record in [v.dropna().to_dict() for k,v in df_merge.iterrows()]]  # 
        if len(updates) > 0:
            logger.info(f"Upserting {len(updates)} photo records.")
            result = cleancollection.bulk_write(updates)
        else:
            logger.warning("Could not merge photo data due to empty dataset")

    except Exception as err:
        msg = "Onbekende fout bij het mergen van foto-informatie met melding: " + str(err)
        logger.error(msg)    
        raise Exception(msg) from err

    finally:
        collection.database.client.close()
        cleancollection.database.client.close()

mergeFotoinfo()

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1134, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 311, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/opt/conda/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2062, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/opt/conda/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2098, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 